# Component Selection for Cardiac vs. Breathing Separation
This notebook performs kernel PCA using the chosen sigma value (from the previous notebook)
and displays the time series, FFT, and PSD for each component.
You can also display cine reconstructions for individual components.
Based on these outputs, decide which component indices are cardiac (i.e., do not contain significant respiratory motion).
Record the list of selected components for use in the final pipeline.

### Loading packages and data

In [ ]:
import yaml
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import utils.pca as pca
import utils.gif as gif
import utils.data_ingestion as di
import utils.reconstruction as recon

def load_config(config_file="config.yaml"):
    with open(config_file, "r") as f:
        config = yaml.safe_load(f)
    return config

config = load_config()

# Set the file paths (or load from a temporary config file if you wish)
twix_file = config["data"]["twix_file"]
dicom_folder = config["data"]["dicom_folder"]
scans = di.read_twix_file(twix_file, include_scans=[-1], parse_pmu=False)
kspace = di.extract_image_data(scans)
n_phase_encodes_per_frame = kspace.shape[0] // config["data"]["n_frames"]
extended_pe_lines = config["data"]["extended_pe_lines"]
offset = config["data"]["offset"]

### Performing kernel PCA and displaying components

In [ ]:
# Use the best sigma from the previous notebook
sigma = config["processing"]["sigma"]

# Get frame rate from the DICOM files
framerate, frametime = di.get_dicom_framerate(dicom_folder)

# Perform kernel PCA using the chosen sigma
kpca_model, X_kpca, frame_shape, orig_feature_dim = pca.perform_kernel_pca(
    kspace, n_phase_encodes_per_frame, kernel="rbf", sigma=sigma
)

# Plot kernel PCA components (time series, FFT, PSD)
pca.plot_kernel_pc_time_series_and_fft(X_kpca, sampling_rate=framerate, n_components=10)

# Reconstruct and display cine movies for each component.
for comp in range(10):
    recon_component = recon.reconstruct_kspace_with_selected_components_kernel_kspace(
        kpca_model, X_kpca, comp, frame_shape, orig_feature_dim
    )
    recon_cine = recon.direct_ifft_reconstruction(recon_component, extended_pe_lines, offset, True)
    print(f"Displaying reconstruction using component {comp}:")
    display(gif.display_images_as_gif(recon_cine, interval=100, notebook=True))
    
# Based on the outputs, decide which components represent cardiac motion.
# (E.g., selected_components = [2, 3, 4, 5, 6, 7, 8, 9])